In [ ]:
rf_model = RandomForestRegressor(random_state=RANDOM_SEED)

# Flatten your input data (Reshape trainX and testX to 2D for sklearn) without the first feature (target)
trainX_flat = trainX[:, :, 1:].reshape(trainX.shape[0], -1).numpy()
testX_flat = testX[:, :, 1:].reshape(testX.shape[0], -1).numpy()
# numpy array needed for scikit-learn
trainY_arr = trainY.numpy().flatten()

start_time = time.time()
rf_model.fit(trainX_flat, trainY_arr)
training_time = time.time() - start_time

trainY_pred_rf = rf_model.predict(trainX_flat)

***

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


ModuleNotFoundError: No module named 'sklearn'

In [8]:
# Erstellen von Beispieldaten
np.random.seed(42)
dates = pd.date_range(start='2023-01-01', periods=1000, freq='h')
data = pd.DataFrame({
    'Tag': dates.day,
    'Stunde': dates.hour,
    'availableBikeNumber': np.random.randint(0, 50, size=(1000,)),
    'Temperatur': np.random.uniform(0, 30, size=(1000,))
})


In [9]:
data

,Tag,Stunde,availableBikeNumber,Temperatur
0,1,0,38,28.010755
1,1,1,28,0.226031
2,1,2,14,6.759984
3,1,3,42,10.960705
4,1,4,7,14.634294
...,...,...,...,...
995,11,11,25,24.758583
996,11,12,33,8.391106
997,11,13,44,28.271611
998,11,14,5,26.200100


In [10]:
# Skaliere die Features
scaler = StandardScaler()
features = ['Tag', 'Stunde', 'availableBikeNumber', 'Temperatur']
scaled_features = scaler.fit_transform(data[features])


NameError: name 'StandardScaler' is not defined

In [ ]:

# Erstellen der Ziel- und Eingabedaten
X, y = [], []
for i in range(len(data) - 29):  # 24 Stunden History + 5 Stunden Forecast
    X.append(scaled_features[i:i + 24])  # Nutze die letzten 24 Stunden als Eingabe
    y.append(scaled_features[i + 24:i + 29, 2])  # Predict die nächsten 5 Stunden für availableBikeNumber
X, y = np.array(X), np.array(y)

# Aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training des Random Forest Regressors
model = RandomForestRegressor(n_estimators=100, random_state=42)
X_train_flat = X_train.reshape(X_train.shape[0], -1)  # Flatten der 3D Eingaben für das Training
model.fit(X_train_flat, y_train)

# Testing
X_test_flat = X_test.reshape(X_test.shape[0], -1)
y_pred = model.predict(X_test_flat)

# Zeige die Testdaten und Vorhersagen
print("Testdaten (Echte Werte):")
print(scaler.inverse_transform(np.column_stack((X_test[:, -1, :], y_test)))[:10])  # Zeigt letzte Stunde von X_test und y_test

print("\nVorhergesagte Werte:")
print(scaler.inverse_transform(np.column_stack((X_test[:, -1, :], y_pred)))[:10])

# Berechne den MSE
mse = mean_squared_error(y_test, y_pred)
print(f"\nMean Squared Error: {mse}")